# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-11 00:23:36] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32787, mem_fraction_static=0.836, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=378348305, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gp

[2025-06-11 00:23:36] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-11 00:23:47] Attention backend not set. Use flashinfer backend by default.
[2025-06-11 00:23:47] Init torch distributed begin.


[2025-06-11 00:23:47] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-11 00:24:01] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.27s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.27s/it]

[2025-06-11 00:24:03] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=35.52 GB, mem usage=-15.38 GB.
[2025-06-11 00:24:03] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-11 00:24:03] Memory pool end. avail mem=35.20 GB


[2025-06-11 00:24:04] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=35.42 GB
[2025-06-11 00:24:04] INFO:     Started server process [1829368]
[2025-06-11 00:24:04] INFO:     Waiting for application startup.
[2025-06-11 00:24:04] INFO:     Application startup complete.
[2025-06-11 00:24:04] INFO:     Uvicorn running on http://127.0.0.1:32787 (Press CTRL+C to quit)


[2025-06-11 00:24:05] INFO:     127.0.0.1:35352 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-11 00:24:05] INFO:     127.0.0.1:35362 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-11 00:24:05] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-11 00:25:09] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-11 00:25:40] INFO:     127.0.0.1:35372 - "POST /generate HTTP/1.1" 200 OK
[2025-06-11 00:25:40] The server is fired up and ready to roll!


[2025-06-11 00:25:43] INFO:     127.0.0.1:35388 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-11 00:25:43] Child process unexpectedly failed with exitcode=9. pid=1830515
[2025-06-11 00:25:43] Child process unexpectedly failed with exitcode=9. pid=1829911


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-11 00:25:54] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=33119, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=605687511, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_

[2025-06-11 00:25:54] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-11 00:26:07] Attention backend not set. Use flashinfer backend by default.
[2025-06-11 00:26:07] Init torch distributed begin.


[2025-06-11 00:26:08] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.39s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.39s/it]

[2025-06-11 00:26:16] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-06-11 00:26:16] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-11 00:26:16] Memory pool end. avail mem=53.92 GB


[2025-06-11 00:26:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.10 GB
[2025-06-11 00:26:17] INFO:     Started server process [1836656]
[2025-06-11 00:26:17] INFO:     Waiting for application startup.
[2025-06-11 00:26:17] INFO:     Application startup complete.
[2025-06-11 00:26:17] INFO:     Uvicorn running on http://127.0.0.1:33119 (Press CTRL+C to quit)


[2025-06-11 00:26:17] INFO:     127.0.0.1:53998 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-11 00:26:18] INFO:     127.0.0.1:54014 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-11 00:26:18] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:26:21] INFO:     127.0.0.1:54016 - "POST /generate HTTP/1.1" 200 OK
[2025-06-11 00:26:21] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-11 00:26:22] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:26:23] INFO:     127.0.0.1:43642 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-11 00:26:23] Child process unexpectedly failed with exitcode=9. pid=1837038
[2025-06-11 00:26:23] Child process unexpectedly failed with exitcode=9. pid=1836814


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-11 00:26:33] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=30431, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=457179715, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None

[2025-06-11 00:26:44] Casting torch.bfloat16 to torch.float16.


[2025-06-11 00:26:44] Casting torch.bfloat16 to torch.float16.
[2025-06-11 00:26:44] Attention backend not set. Use flashinfer backend by default.
[2025-06-11 00:26:44] Init torch distributed begin.


[2025-06-11 00:26:45] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-11 00:27:05] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=40.93 GB


[2025-06-11 00:27:05] INFO:     Started server process [1839300]
[2025-06-11 00:27:05] INFO:     Waiting for application startup.
[2025-06-11 00:27:05] INFO:     Application startup complete.
[2025-06-11 00:27:05] INFO:     Uvicorn running on http://127.0.0.1:30431 (Press CTRL+C to quit)


[2025-06-11 00:27:06] INFO:     127.0.0.1:43900 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-11 00:27:06] INFO:     127.0.0.1:43912 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-11 00:27:06] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:27:09] INFO:     127.0.0.1:43928 - "POST /generate HTTP/1.1" 200 OK
[2025-06-11 00:27:09] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-11 00:27:11] Detected chat template content format: string
[2025-06-11 00:27:11] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:27:11] INFO:     127.0.0.1:40618 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-11 00:27:11] Child process unexpectedly failed with exitcode=9. pid=1839917
[2025-06-11 00:27:11] Child process unexpectedly failed with exitcode=9. pid=1839740


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-11 00:27:23] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=37458, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=893823905, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gp

[2025-06-11 00:27:35] Casting torch.bfloat16 to torch.float16.


[2025-06-11 00:27:36] Casting torch.bfloat16 to torch.float16.
[2025-06-11 00:27:36] Attention backend not set. Use flashinfer backend by default.
[2025-06-11 00:27:36] Init torch distributed begin.


[2025-06-11 00:27:36] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.17it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.17it/s]

[2025-06-11 00:27:53] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=58.34 GB, mem usage=1.77 GB.
[2025-06-11 00:27:53] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-06-11 00:27:53] Memory pool end. avail mem=58.26 GB


[2025-06-11 00:27:54] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.94 GB


[2025-06-11 00:27:54] INFO:     Started server process [1841623]
[2025-06-11 00:27:54] INFO:     Waiting for application startup.
[2025-06-11 00:27:54] INFO:     Application startup complete.
[2025-06-11 00:27:54] INFO:     Uvicorn running on http://127.0.0.1:37458 (Press CTRL+C to quit)
[2025-06-11 00:27:54] INFO:     127.0.0.1:60234 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-11 00:27:55] INFO:     127.0.0.1:60248 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-11 00:27:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:27:58] INFO:     127.0.0.1:60258 - "POST /generate HTTP/1.1" 200 OK
[2025-06-11 00:27:58] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-11 00:28:00] Detected chat template content format: string
[2025-06-11 00:28:00] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:28:00] INFO:     127.0.0.1:60260 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-11 00:28:00] Child process unexpectedly failed with exitcode=9. pid=1841910
[2025-06-11 00:28:00] Child process unexpectedly failed with exitcode=9. pid=1841772


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-11 00:28:12] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=33697, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=140497839, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info'

[2025-06-11 00:28:23] Attention backend not set. Use flashinfer backend by default.
[2025-06-11 00:28:23] Init torch distributed begin.
[2025-06-11 00:28:23] Init torch distributed ends. mem usage=0.00 GB


[2025-06-11 00:28:23] Load weight begin. avail mem=78.50 GB


[2025-06-11 00:28:24] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-06-11 00:28:30] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-11 00:28:31] INFO:     Started server process [1842647]
[2025-06-11 00:28:31] INFO:     Waiting for application startup.
[2025-06-11 00:28:31] INFO:     Application startup complete.
[2025-06-11 00:28:31] INFO:     Uvicorn running on http://0.0.0.0:33697 (Press CTRL+C to quit)


[2025-06-11 00:28:31] INFO:     127.0.0.1:47670 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-11 00:28:32] INFO:     127.0.0.1:47680 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-11 00:28:32] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:28:33] INFO:     127.0.0.1:47684 - "POST /generate HTTP/1.1" 200 OK
[2025-06-11 00:28:33] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-11 00:28:36] Detected chat template content format: string
[2025-06-11 00:28:36] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-11 00:28:37] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, accept len: 1.69, cuda graph: False, gen throughput (token/s): 10.54, #queue-req: 0


[2025-06-11 00:28:37] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, accept len: 1.62, cuda graph: False, gen throughput (token/s): 104.15, #queue-req: 0


[2025-06-11 00:28:38] Decode batch. #running-req: 1, #token: 220, token usage: 0.01, accept len: 1.68, cuda graph: False, gen throughput (token/s): 109.09, #queue-req: 0


[2025-06-11 00:28:38] Decode batch. #running-req: 1, #token: 282, token usage: 0.01, accept len: 1.55, cuda graph: False, gen throughput (token/s): 97.51, #queue-req: 0


[2025-06-11 00:28:39] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, accept len: 1.77, cuda graph: False, gen throughput (token/s): 112.87, #queue-req: 0


[2025-06-11 00:28:40] Decode batch. #running-req: 1, #token: 421, token usage: 0.02, accept len: 1.70, cuda graph: False, gen throughput (token/s): 108.29, #queue-req: 0


[2025-06-11 00:28:40] Decode batch. #running-req: 1, #token: 490, token usage: 0.02, accept len: 1.73, cuda graph: False, gen throughput (token/s): 113.14, #queue-req: 0


[2025-06-11 00:28:41] INFO:     127.0.0.1:47690 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-11 00:28:41] Child process unexpectedly failed with exitcode=9. pid=1842927
[2025-06-11 00:28:41] Child process unexpectedly failed with exitcode=9. pid=1842793


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).